In [ ]:
import random
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
probabilities = np.logspace(-2, -0.046, num=10)
neighbors = [4, 20, 50]
graph_types = ["er", "ws"]

In [ ]:
er_graphs = []
ws_graphs01 = []
ws_graphs02 = []
ws_graphs03 = []

In [ ]:
def compute_results(prob, nodes, graph_type, neighbor):
  file = open("output.txt", "r")
  for lines in file:
    results = lines.split(" ")
    inf_lim = int(results[0])
    sup_lim = int(results[1].removesuffix("\n"))

    if (graph_type == graph_types[0]):
      er_graphs.append({prob: [nodes, inf_lim, sup_lim]})
    elif (neighbor == neighbors[0]):
      ws_graphs01.append({prob: [nodes, inf_lim, sup_lim]})
    elif (neighbor == neighbors[1]):
      ws_graphs02.append({prob: [nodes, inf_lim, sup_lim]})
    else:
      ws_graphs03.append({prob: [nodes, inf_lim, sup_lim]})

In [ ]:
for prob in probabilities:
    for i in range(10):
        nodes = random.randint(900, 1100)
        for graph_type in graph_types:
            if graph_type == graph_types[0]:
                subprocess.run(["./main", f"{prob}", f"{nodes}", f"{graph_type}"])
                compute_results(prob, nodes, graph_type, -1)
            if graph_type == graph_types[1]:
                for neighbor in neighbors:
                    subprocess.run(["./main", f"{prob}", f"{nodes}", f"{graph_type}", f"{neighbor}"])
                    compute_results(prob, nodes, graph_type, neighbor)

In [ ]:
def list_to_dataframe(list, index):
    data = []
    for item in list:
        for key, values in item.items():
            data.append([key, values[index]])
    return pd.DataFrame(data, columns=['Key', 'Value'])

In [ ]:
def make_table(first_data_dict, second_data_dict, third_data_dict, fourth_data_dict):
    df = pd.DataFrame({
        'Probabilities': probabilities,
        'Erdos-Renyi': [first_data_dict.get(key, 0) for key in probabilities],
        'Watts-Strogatz K=4': [second_data_dict.get(key, 0) for key in probabilities],
        'Watts-Strogatz K=20': [third_data_dict.get(key, 0) for key in probabilities],
        'Watts-Strogatz K=50': [fourth_data_dict.get(key, 0) for key in probabilities]
    })

    df = df.round(2)

    return df

In [ ]:
def plot(fisrt_data_dict, second_data_dict, third_data_dict, fourth_data_dict, y_label):
    er_data = [fisrt_data_dict.get(key, 0) for key in probabilities]
    ws_data01 = [second_data_dict.get(key, 0) for key in probabilities]
    ws_data02 = [third_data_dict.get(key, 0) for key in probabilities]
    ws_data03 = [fourth_data_dict.get(key, 0) for key in probabilities]

    x = np.arange(len(probabilities))

    plt.plot(x, er_data, label=f"Erdos-Renyi")
    plt.plot(x, ws_data01, label=f"Watts-Strogatz K = 4")
    plt.plot(x, ws_data02, label=f"Watts-Strogatz K = 20")
    plt.plot(x, ws_data03, label=f"Watts-Strogatz K = 50")

    plt.xlabel(f"Probabilidades")
    plt.ylabel(f"{y_label}")
    plt.title(f"Erdos-Renyi X Watts-Strogatz")

    formatted_probabilities = [f"{prob:.2f}" for prob in probabilities]
    plt.xticks(x, formatted_probabilities)

    
    plt.legend()

    plt.show()

In [ ]:
inf_lim_er = list_to_dataframe(er_graphs, 1)
inf_lim_ws01 = list_to_dataframe(ws_graphs01, 1)
inf_lim_ws02 = list_to_dataframe(ws_graphs02, 1)
inf_lim_ws03 = list_to_dataframe(ws_graphs03, 1)

sup_lim_er = list_to_dataframe(er_graphs, 2)
sup_lim_ws01 = list_to_dataframe(ws_graphs01, 2)
sup_lim_ws02 = list_to_dataframe(ws_graphs02, 2)
sup_lim_ws03 = list_to_dataframe(ws_graphs03, 2)

In [ ]:
mean_of_inf_lim_er = inf_lim_er.groupby('Key')['Value'].mean()
mean_of_inf_lim_ws01 = inf_lim_ws01.groupby('Key')['Value'].mean()
mean_of_inf_lim_ws02 = inf_lim_ws02.groupby('Key')['Value'].mean()
mean_of_inf_lim_ws03 = inf_lim_ws03.groupby('Key')['Value'].mean()

df = make_table(mean_of_inf_lim_er, mean_of_inf_lim_ws01, mean_of_inf_lim_ws02, mean_of_inf_lim_ws03)
df.to_csv('limites_inferiores.csv', index=False)

plot(mean_of_inf_lim_er, mean_of_inf_lim_ws01, mean_of_inf_lim_ws02, mean_of_inf_lim_ws03, 'Média dos limites inferiores cromáticos')

No Gráfico 1, a primeira característica que se pode perceber ao analisar esse gráfico é o crescimento exponencial do limite inferior do número cromático para os grafos de Erdos-Renyi. Isso ocorre pois com o aumento do valor da probabilidade surgem mais cliques de grande tamanho, o que aumenta o número de cores necessárias para garantir que os vértices tenham todos cores diferentes. Já para os grafos de Watts-Strogatz percebe-se inicialmente que, quanto maior o valor do número inicial de vizinhos K, maior é o limite inferior do número cromático. Isso ocorre porque um valor maior de K implica que cada vértice é inicialmente conectado a mais vizinhos, resultando em um grafo mais denso. Essa densidade exige um maior número de cores para garantir que vértices adjacentes não compartilhem a mesma cor. Além disso, também é possível notar que o limite inferior diminui à medida que a probabilidade aumenta nos grafos de Watts-Strogatz, um efeito praticamente inverso ao que acontece nos grafos de Erdos-Renyi. Essa redução no limite inferior ocorre porque o aumento da probabilidade leva o grafo de uma estrutura mais regular e clusterizada para uma configuração mais aleatória.

In [ ]:
stddev_of_inf_lim_er = inf_lim_er.groupby('Key')['Value'].std()
stddev_of_inf_lim_ws01 = inf_lim_ws01.groupby('Key')['Value'].std()
stddev_of_inf_lim_ws02 = inf_lim_ws02.groupby('Key')['Value'].std()
stddev_of_inf_lim_ws03 = inf_lim_ws03.groupby('Key')['Value'].std()

plot(stddev_of_inf_lim_er, stddev_of_inf_lim_ws01, stddev_of_inf_lim_ws02, stddev_of_inf_lim_ws03, 'Desvio padrão dos limites inferiores cromáticos')

No Gráfico 2 é difícil reconhecer algum padrão, já que as linhas geradas não apresentam nenhum comportamento constante perceptível. Entretanto, é possível notar que, para a maioria das probabilidades, o desvio padrão é maior quanto mais alto o número inicial de vizinhos K nos grafos de Watts-Strogatz. Em geral os valores de desvio padrão se mantiveram baixos para os dois modelos.

In [ ]:
mean_of_sup_lim_er = sup_lim_er.groupby('Key')['Value'].mean()
mean_of_sup_lim_ws01 = sup_lim_ws01.groupby('Key')['Value'].mean()
mean_of_sup_lim_ws02 = sup_lim_ws02.groupby('Key')['Value'].mean()
mean_of_sup_lim_ws03 = sup_lim_ws03.groupby('Key')['Value'].mean()

df = make_table(mean_of_sup_lim_er, mean_of_sup_lim_ws01, mean_of_sup_lim_ws02, mean_of_sup_lim_ws03)
df.to_csv('limites_superiores.csv', index=False)

plot(mean_of_sup_lim_er, mean_of_sup_lim_ws01, mean_of_sup_lim_ws02, mean_of_sup_lim_ws03, 'Média dos limites superiores cromáticos')

Ao analisar o Gráfico 3, a primeira característica que se pode notar é, assim como ocorreu com o limite inferior, o limite superior nos grafos de Erdos-Renyi cresce de forma exponencial à medida que o valor da probabilidade aumenta, podendo alcançar valores pŕoximos ao número N de vértices do grafo quando a probabilidade se aproxima de 1. Isso ocorre porque, à medida que a probabilidade cresce, o grafo se torna cada vez mais denso, com mais arestas entre os vértices, se aproximando cada vez mais de um grafo completo. Já em relação aos grafos de Watts-Strogatz, nota-se que, assim como no gráfico dos limites inferiores, o limite superior do número cromático é maior quanto mais alto o valor do número inicial de vizinhos K, o que está diretamente relacionado à densidade de conexões locais no grafo. Além disso, apesar de ser quase imperceptível no gráfico, na Tabela 2 podemos notar que independente do valor de K, o limite superior aumenta à medida que a probabilidade também aumenta. Isso pode ser explicado porque, com o aumento da aleatoriedade, aumenta a conectividade global e a densidade de adjacências entre nós, o que torna necessário um maior número de cores para lidar com essas novas conexões.

In [ ]:
stddev_of_sup_lim_er = sup_lim_er.groupby('Key')['Value'].std()
stddev_of_sup_lim_ws01 = sup_lim_ws01.groupby('Key')['Value'].std()
stddev_of_sup_lim_ws02 = sup_lim_ws02.groupby('Key')['Value'].std()
stddev_of_sup_lim_ws03 = sup_lim_ws03.groupby('Key')['Value'].std()

plot(stddev_of_sup_lim_er, stddev_of_sup_lim_ws01, stddev_of_sup_lim_ws02, stddev_of_sup_lim_ws03, 'Desvio padrão dos limites superiores cromáticos')

Já no Gráfico 4, nota-se que, enquanto o modelo de Watts-Strogatz mantém o desvio padrão baixo para os limites superiores (assim como ocorreu com os limites inferiores), o modelo de Erdos-Renyi apresenta um crescimento exponencial do desvio padrão, muito maior do que os valores apresentados para o desvio padrão dos limites inferiores. Isso ocorre pois o limite superior do número cromático em um grafo depende do grau máximo, que tende a variar em grafos de Erdos-Renyi devido à aleatoriedade do modelo, de modo que, quanto maior a probabilidade, maior a variação.